<left>
    <img src="https://kpi.ua/files/images/kpi.png" width="300" alt="kpi logo"  />
</left>

## Денисенко Ілля КМ-82

## Курсова робота на тему "Метод випадкового пошуку з постійним радіусом пошуку"

## Outline

* Abstract
* Introduction
* Materials and Methods 
* Solving the Problem 
  * Importing Libraries/defining auxiliary functions
  * Defining constants
  * Function(s) defenition
  * Calculations
  * Displaying results 
* Conclusions. 
* Authors
* References 



## Abstract

Ця курсова робота присвячена оптимізації функцій багатьох змінних численими методами. \
Для прикладу ми вибрали функцію Розенброка та метод випадкового пошуку з постіним радіусом пошуку.



## Introduction


Метод випадкового пошуку є методом нульового порядку тому можливо не є самим оптимальним методом, проте він доволі простий в імплементації та не робить особливих припущень про природу функції тому може бути використаний у ситуаціях де не можуть бути використані більш просунуті методи, або навіть може бути використаний як елемент інших методів. \
Як елементи для цього методу ми використаємо одномірний пошук з методом золотого перетину та ДСК-Пауела. Інтервал невизначеності ми знайдемо за допомогою алгоритму Свена. \
Метод випадкового пошуку залежить від багатьох коефіцієнтів, тому метою цієї роботи буде визначити як вони впливають на роботу методу та визначити оптимальні коєфіцієнти.

## Materials and Methods 


## Solving the problem


Функція Розенброка використовується для перевірки алгоритмів оптимізаціїї і має мінімум 0 у точці (1, 1). \
Щоб перевірити як часто метод викликає цю функцію ми заведемо лічильник і будемо інкрементувати його на кожен виклик. (Для цього був використаний класс з методом `__call__()` на мові програмування Python). 

На вибір з методів одномірного пошуку ми використаємо метод золотого перетину, що працює подібно методу дихотомії, але обчислює функцію меншу кількість раз через особливість пропорції та метод ДСК-Паула що є комбінацією метода ДСК та квадратичної апроксимації.

Для умовної оптимізації ми використаємо метод штрафних функцій внутрішньої точки.

----

## Import Libraries/Define Auxillary Functions

In [6]:
from typing import Tuple, Optional, NoReturn, Sequence, Iterable, Any
from collections.abc import Callable
from dataclasses import dataclass
import math
import random
import functools


# ====================
# Convinient definition
# ====================

Function = Callable[['Vec2'], float]


def trace(f):
    VERBOSE = False
    DEBUG = False

    def wrapper(*args, **kwargs):
        res = f(*args, **kwargs)
        if not DEBUG:
            return res
        else:
            if VERBOSE:
                params = ', '.join(map(str, args))
                keywords = "" if len(kwargs) == 0 else str(kwargs)
                print(
                    f"{f.__name__}({params}, {keywords}) =\n {res}"
                )
            else:
                print(f"{f.__name__}() = {res}")

            return res
    return wrapper


def panic(msg: str) -> NoReturn:
    """Raise RuntimeError with given message"""
    raise RuntimeError(msg)


def debug(*args, **kwargs) -> None:
    """Just a fancy keyword for a print()"""
    print(*args, **kwargs)


def avg(iterable):
    """Returns mean of sequence of `-inf` if empty"""
    if len(iterable) == 0:
        return float("-inf")
    sum_all = sum(iterable)
    n = len(iterable)
    return sum_all / n


def moda(xs):
    """Returns average value with removing anomaly big values"""
    data = [min(xs)]
    values = sorted(xs)[1:]
    for x in values:
        if abs(x - avg(data)) < 100:
            data.append(x)
    return avg(data)


def show_results(
        results,
        times,
        extremums = None
) -> None:
    print("==== Results =====")
    min_res = min(results)
    max_res = max(results)
    avg_res = avg(results)
    moda_res = moda(results)
    print(
        f"Function results.\n"
        f"min: {min_res:.2e}, moda: {moda_res:.2e}, avg: {avg_res:.2e}, max: {max_res:.2e}")
    if extremums is not None:
        min_extr = min(extremums)
        max_extr = max(extremums)
        print(
            f"Extremums.\n"
            f"min: {min_extr}, max: {max_extr}"
        )
    min_time = min(times)
    max_time = max(times)
    print(
        f"Function calculation. [{min_time:.2e} : {max_time:.2e}]")


@dataclass(frozen=True, eq=True)
class Vec2:
    pair: Tuple[float, float]

    def add(self, other: 'Vec2') -> 'Vec2':
        """x + y"""
        x1, x2 = self.pair
        y1, y2 = other.pair
        return point(x1 + y1, x2 + y2)

    def sub(self, other: 'Vec2') -> 'Vec2':
        """x - y"""
        return self + other * -1

    def times(self, k: float) -> 'Vec2':
        """k * x"""
        x1, x2 = self.pair
        return point(x1 * k, x2 * k)

    def __lt__(self, other: 'Vec2') -> bool:
        return self.norm() < other.norm()

    def __add__(self, other: 'Vec2') -> 'Vec2':
        return self.add(other)

    def __sub__(self, other: 'Vec2') -> 'Vec2':
        return self.sub(other)

    def __mul__(self, k: float) -> 'Vec2':
        return self.times(k)

    def __repr__(self):
        x, y = self.pair
        return f"({x:.5f}, {y:.5f})"

    def norm(self) -> float:
        """||x||"""
        x1, x2 = self.pair
        return math.sqrt(x1 ** 2 + x2 ** 2)


def point(x1: float, x2: float) -> Vec2:
    """Helper function to conver x1, x2 to Vec2"""
    return Vec2((x1, x2))


## Defining constants

In [2]:
# ====================
# Constants
# ====================
X0 = point(50, 50)
K = 1e-5  # swen step coefficient
STEP_NUM = 1_000  # swen max step number
GOLDEN_EPSILON = 0.01  # precision of golden_ratio
DSK_EPSILON = 1e-4  # precision of dsk_powell
EPSILON = 1e-5  # presicion of optimization
RADIUS = 0.001  # radius of random search
N = 10  # number of random tries in random search


## Function(s) defenition

In [3]:
def lstep(xi: Vec2, si: Vec2, k: float) -> float:
    """Returns step for Swen alorithm"""
    return k * xi.norm() / si.norm()


def choose_direction(f: Function, x0: Vec2, s: Vec2, d: float) -> Optional[float]:
    """Direction for Swen algorithm

    Calculate function in [x0 - step, x + step]
    Returns sign() of direction needed to minimize a function
    """

    delta = lstep(x0, s, d)

    left = f(x0 - s * delta)
    exact = f(x0)
    right = f(x0 + s * delta)

    if left > exact and exact > right:
        return 1.0
    elif left < exact and exact < right:
        return -1.0
    else:  # if left == right
        return None


@trace
def swen_interval(f: Function, x0: Vec2, s: Vec2, d: float) -> Tuple[float, float, float]:
    """Base interval for use with golden-ratio or DSK-Powell

    Returns (left, approx_min, right)"""
    step = lstep(x0, s, d)
    direction = choose_direction(f, x0, s, d)
    if direction is None:
        return (-step, 0, step)

    k = 0
    way = [
        (0.0, f(x0))
    ]
    while k < STEP_NUM:
        previous_point, fp = way[-1]

        next_point = previous_point + direction * step * (2 ** k)
        fn = f(x0 + s * next_point)
        if fn > fp:
            middle_point = (next_point + previous_point) / 2
            fm = f(x0 + s * middle_point)

            if fm < fp:
                return (previous_point, middle_point, next_point)
            else:
                if len(way) < 2:
                    panic("tried to get 2th point from the end, "
                          "while way in swen_alogrithm has only one value")
                next_previos_point, _ = way[-2]
                return (next_previos_point, previous_point, middle_point)
        else:
            k += 1
            way.append((next_point, fn))
    panic(f"more than {STEP_NUM} steps in swen algorithm")


@trace
def golden_ratio(
        f: Function,
        interval: Tuple[float, float],
        x: Vec2,
        s: Vec2,
        eps: float,
) -> float:
    """Returns way length to go to minimize a function `f`

    Finds extremum of `f` on `interval` by searching with `s` direction"""
    left, right = interval

    while abs(left - right) > eps:
        length = abs(left - right)
        x1 = left + 0.382 * length
        x2 = left + 0.618 * length

        f1 = f(x + s * x1)
        f2 = f(x + s * x2)

        if f1 < f2:
            left, right = left, x1
        else:
            left, right = x1, right
    # add noise to avoid zero paths
    return left + eps


@trace
def dsk_powell(
        f: Function,
        interval: Tuple[float, float, float],
        x0: Vec2,
        s: Vec2,
        eps: float,
) -> float:
    """Returns way length to go to minimize a function `f`

    Finds extremum of `f` on `interval` by quadratic approximation"""

    def final(x2: float, x_star: float, f2: float, f_star: float):
        first = abs(x2 - x_star)
        second = abs(f2 - f_star)
        return first < eps and second < eps

    x1, x2, x3 = interval
    while True:
        # compute functions
        f1 = f(x0 + s * x1)
        f2 = f(x0 + s * x2)
        f3 = f(x0 + s * x3)

        a1 = (f2 - f1) / (x2 - x1)
        a2 = (1 / (x3 - x1)) * \
            ((f3 - f2) / (x3 - x2) - ((f2 - f1) / (x2 - x1)))

        x_star = (x1 + x2) / 2 - a1 / (2 * a2)
        f_star = f(x0 + s * x_star)

        if final(x2, x_star, f2, f_star):
            return x_star
        else:
            x_min, _ = min(
                (x1, f1),
                (x2, f2),
                (x3, f3),
                (x_star, f_star),
                key=lambda pair: pair[1])
            left = filter(lambda x: x < x_min and x != x_min,
                          (x1, x2, x3, x_star))
            right = filter(lambda x: x > x_min and x != x_min,
                           (x1, x2, x3, x_star))
            x1 = max(left)
            x2 = x_min
            x3 = min(right)


@trace
def random_try(f: Function, x: Vec2, r: float, n: int) -> Vec2:
    """Return direction to search for next minimum

    Tries points in the radius `r` around `x` and find minimum of `f` for these tries"""
    acc = 0.0
    tries = []

    @trace
    def try_point(dot):
        si = point(math.sin(dot), math.cos(dot))
        xi = x + si * r
        fi = f(xi)
        tries.append((si, fi))
        return (si, fi)

    while acc < math.pi:
        acc += math.pi / n
        try_point(acc + random.uniform(0, math.pi/n))
    s, _ = min(tries, key=lambda pair: pair[1])  # type: ignore
    return s


@trace
def satisfactory_result(
    next_point: Vec2,
    point: Vec2,
    fk1: float,
    fk: float,
    eps: float,
) -> bool:
    delta_x = (next_point - point).norm()  # / point.norm()
    delta_f = abs((fk1 - fk))  # / abs(fk)
    return delta_x < eps and delta_f < eps


@trace
def minimize_with_random_search(
        f: Function,
        x0: Vec2,
        *,
        way_search: str,  # Literal["golden_ratio", "dsk"]
        d_swen: float = K,
        dsk_eps: float = DSK_EPSILON,
        golden_eps: float = GOLDEN_EPSILON,
        n: int = N,
        radius: float = RADIUS,
        eps: float = EPSILON,
) -> Vec2:
    """Minimize function `f` with method of random search and constant step"""
    search = True
    xi = x0
    fi = f(x0)
    while search:
        s = random_try(f, xi, radius, n)
        interval = swen_interval(f, xi, s, d_swen)
        lmbd = 0.0
        if way_search == "golden_ratio":
            left, _, right = interval
            lmbd = golden_ratio(f, (left, right), xi, s, golden_eps)
        elif way_search == "dsk":
            lmbd = dsk_powell(f, interval, xi, s, dsk_eps)
        xn = xi + s * lmbd
        fn = f(xn)
        if satisfactory_result(xn, xi, fn, fi, eps):
            search = False
        else:
            xi = xn
            fi = fn
    return xi


def minimize_with_condition(
        f: Function,
        x0: Vec2,
        condition: Callable,
        *,
        d_swen: float = K,
        dsk_eps: float = DSK_EPSILON,
        golden_eps: float = GOLDEN_EPSILON,
        n: int = N,
        radius: float = RADIUS,
        eps: float = EPSILON,
) -> Vec2:
    """Minimize function `f` with condition

    Uses method of random search with constant step to find minimum
    Uses method of penalty functions to fit conditions
    """
    r = 1.0
    def px(r): return lambda x: f(x) + r * condition(x) ** 2
    approx = minimize_with_random_search(
        px(r),
        x0,
        way_search="dsk",
        d_swen=d_swen,
        dsk_eps=dsk_eps,
        n=n,
        radius=radius,
        eps=eps
    )
    print(f"r={r}, approx={approx}")
    while condition(approx) < -0.01:
        r *= 10.0
        approx = minimize_with_random_search(
            px(r),
            x0,
            way_search="dsk",
            d_swen=d_swen,
            dsk_eps=dsk_eps,
            n=n,
            radius=radius,
            eps=eps
        )
        print(f"r={r}, approx={approx}")
    return approx


## Calculations

In [4]:
class Rozenbrok:
    counter: int

    def __init__(self):
        self.counter = 0

    def __repr__(self):
        return f"f-{self.counter}"

    @functools.lru_cache(128)
    def calculate(self, point: Vec2) -> float:
        self.counter += 1
        A = 1
        B = 100
        x, y = point.pair
        return (A - x) ** 2 + B * (y - x ** 2) ** 2

    def __call__(self, point: Vec2) -> float:
        #import time
        # time.sleep(0.05)

        return self.calculate(point)


def test_way_search():
    # test data
    start = point(50, 50)
    d_swen = 0.01
    n = 9
    radius = 0.001
    # placeholders, filled later
    dsk_eps = 0.0
    golden_eps = 0.0
    eps = 0.0
    print(
        "\n== "
        "Testing how function calculation varies depending on "
        "method for way seach (golden_ratio or dsk_powell) =="
    )
    print(
        f"d_swen={d_swen}\n"
        f"start={start}\n"
        f"radius={radius}\n"
        f"n={n}"
    )
    methods = ["golden_ratio", "dsk"]
    for way_search in methods:
        if way_search == "golden_ratio":
            golden_eps = 0.01
            eps = 0.001
            print(f"\nUsing GoldenRatio\n"
                  f"golden_eps={golden_eps}\n"
                  f"eps={eps}")
        elif way_search == "dsk":
            eps = 1e-09
            dsk_eps = 0.000_1
            print(f"\nUsing DSK-Powell\n"
                  f"dsk_eps={dsk_eps}\n"
                  f"eps={eps}")
        results = []
        times = []
        tests = 10
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                golden_eps=golden_eps,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
        show_results(results, times)
    print(
        ">\n"
        "Using DSK-Powell method for optimal path"
        " allows us to get much more precise values"
    )


def test_starts():
    d_swen = 0.001
    way_search = "dsk"
    dsk_eps = 0.000_1
    n = 10
    radius = 0.001
    eps = 1e-09
    print("==== Testing how function calculation varies depending on starting point ====")
    print(
        f"d_swen={d_swen}\n"
        f"way_search={way_search}\n"
        f"dsk_eps={dsk_eps}\n"
        f"n={n}\n"
        f"radius={radius}\n"
        f"eps={eps}\n"
    )
    tests = 20
    results = []
    times = []
    for i in range(tests):
        f = Rozenbrok()
        # add epsilon to avoid zero division
        start = point(1.0, 1.0) * (10 * (i - 10) + eps)
        print(
            f"Starting at {start}"
        )
        extremum = minimize_with_random_search(
            f,
            start,
            way_search=way_search,
            d_swen=d_swen,
            dsk_eps=dsk_eps,
            n=n,
            radius=radius,
            eps=eps,
        )

        minimum = f(extremum)
        count = f.counter
        times.append(f.counter)
        results.append(minimum)
        print(f"min = {minimum:.2e} with function calculated {count} times")
    show_results(results, times)
    print(
        ">\n"
        "Calculation time doesn't seem to correlate with starting point"
    )


def test_golden_epsilon():
    start = point(50, 50)
    d_swen = 0.01
    way_search = "golden_ratio"
    n = 3
    radius = 1e-05
    eps = 0.001
    print("")
    print("== Testing how function calculation varies depending on `e` in golden_ratio ==")
    print(
        f"start={start}\n"
        f"d_swen={d_swen}\n"
        f"way_search={way_search}\n"
        f"n={n}\n"
        f"radius={radius}\n"
        f"eps={eps}\n"
    )
    epsilons = [0.2, 0.1, 0.05, 0.02, 0.01, 0.001]
    for golden_eps in epsilons:
        print(
            f"Using golden_eps={golden_eps}"
        )
        results = []
        times = []
        tests = 5
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                golden_eps=golden_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
        show_results(results, times)
    print(
        ">\n"
        "Function minimum is more precise with lesser epsilon for golden ratio.\n"
        "(it comes with more of function calculation)\n"
        "Buf when epsilon for golden_ratio coming close to random method epsilon\n"
        "method starts to produce false-positives"
    )


def test_swen_precision():
    start = point(50, 50)
    way_search = "dsk"
    dsk_eps = 0.000_1
    n = 10
    radius = 0.001
    eps = 1e-09
    print(
        "\n== "
        "Testing how function calculation varies depending on "
        "`d` in swen_algorithm =="
    )
    print(
        f"start={start}\n"
        f"dsk_eps={dsk_eps}\n"
        f"way_search={way_search}\n"
        f"n={n}\n"
        f"radius={radius}\n"
        f"eps={eps}\n"
    )
    ds = [0.05, 0.01, 0.001, 0.0001, eps * 0.001]
    for d_swen in ds:
        print(
            f"Using d_swen={d_swen}"
        )
        results = []
        times = []
        tests = 5
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
        show_results(results, times)
    print(
        ">\n"
        "Using lesser `d` for step calculation in swen interval "
        "improves both speed and precision of method\n"
        "But as in case with precision of golden_ratio it produces false-positives "
        "when `d` is lesser than `eps` for random search method"
    )


def test_radius():
    start = point(50, 50)
    d_swen = 1e-05
    way_search = "dsk"
    dsk_eps = 0.000_1
    n = 10
    eps = 1e-09
    print("")
    print(
        "== "
        "Testing how function calculation varies depending on "
        "`radius` of random search =="
    )
    print(
        f"d_swen={d_swen}\n"
        f"start={start}\n"
        f"dsk_eps={dsk_eps}\n"
        f"way_search={way_search}\n"
        f"n={n}\n"
        f"eps={eps}\n"
    )
    rs = [1.0, 0.5, 0.2, 0.1, 0.01, 0.001, 0.000_01, 0.000_000_1]
    for radius in rs:
        print(
            f"Using radius={radius}"
        )
        results = []
        times = []
        tests = 10
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
        show_results(results, times)
    print(
        ">\n"
        "Different radius doestn't seem to correlate with precision much"
    )


def test_tries():
    start = point(50, 50)
    d_swen = 1e-05
    way_search = "dsk"
    dsk_eps = 0.000_1
    radius = 0.001
    eps = 1e-09
    print("")
    print(
        "== "
        "Testing how function calculation varies depending on "
        "number of tries in random search =="
    )
    print(
        f"d_swen={d_swen}\n"
        f"start={start}\n"
        f"way_search={way_search}\n"
        f"dsk_eps={dsk_eps}\n"
        f"radius={radius}\n"
        f"eps={eps}\n"
    )
    ns = [3, 5, 7, 8, 9, 10, 12, 15, 20]
    for n in ns:
        print(
            f"Using n={n}"
        )
        results = []
        times = []
        tests = 10
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
        show_results(results, times)
    print(
        ">\n"
        "More tries produces slightly more precise values"
    )


def test_epsilon():
    start = point(50, 50)
    d_swen = 1e-05
    way_search = "dsk"
    dsk_eps = 0.000_1
    n = 10
    radius = 0.001
    print(
        "\n== "
        "Testing how function calculation varies depending on "
        "epsilon in random search =="
    )
    print(
        f"d_swen={d_swen}\n"
        f"start={start}\n"
        f"way_search={way_search}\n"
        f"dsk_eps={dsk_eps}\n"
        f"radius={radius}\n"
        f"n={n}\n"
    )
    epsilons = [0.000_1, 1e-06, 1e-09, 1e-12]
    for eps in epsilons:
        print(
            f"Using eps={eps}"
        )
        results = []
        times = []
        extremums = []
        tests = 10
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_random_search(
                f,
                start,
                way_search=way_search,
                d_swen=d_swen,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
            extremums.append(extremum)
        show_results(results, times, extremums)
    print(
        ">\n"
        "Lesser epsilon gives more precise calculation, but requires more time"
    )


def test_conditional():
    start = point(50, 50)
    d_swen = 1e-05
    way_search = "dsk"
    dsk_eps = 0.000_1
    n = 10
    radius = 0.001
    eps = 1e-9
    print("Test conditional optimization")
    print(
        f"d_swen={d_swen}\n"
        f"start={start}\n"
        f"way_search={way_search}\n"
        f"dsk_eps={dsk_eps}\n"
        f"radius={radius}\n"
        f"eps={eps}\n"
        f"n={n}\n"
    )

    def condition_with_min_out(point: Vec2) -> float:
        x1, x2 = point.pair
        return -1 - x1 - x2

    def condition_with_min_in(point: Vec2) -> float:
        x1, x2 = point.pair
        return 2 - x1 - x2

    def condition_convex(point: Vec2) -> float:
        x1, x2 = point.pair
        return (x1 - 5) ** 2 - (x2 - 5) ** 2 - 1
    conditions = [
        condition_with_min_in,
        condition_with_min_out,
        condition_convex,
    ]
    for condition in conditions:
        print(
            f"Using condition.__name__={condition.__name__}"
        )
        results = []
        times = []
        extremums = []
        tests = 5
        for _ in range(tests):
            f = Rozenbrok()
            extremum = minimize_with_condition(
                f,
                start,
                condition,
                d_swen=d_swen,
                dsk_eps=dsk_eps,
                n=n,
                radius=radius,
                eps=eps,
            )

            minimum = f(extremum)
            times.append(f.counter)
            results.append(minimum)
            extremums.append(extremum)
        show_results(results, times, extremums)
    print(
        ">\n"
        "Function minimization works faster when local minimum is inside condition area\n"
        "Also random search works faster with flex shapes"
        " rather then with linear restrictions."
    )


## Displaying the results

In [5]:
test_way_search()
test_starts()
test_golden_epsilon()
test_swen_precision()
test_radius()
test_tries()
test_epsilon()
test_conditional()


== Testing how function calculation varies depending on method for way seach (golden_ratio or dsk_powell) ==
d_swen=0.01
start=(50.00000, 50.00000)
radius=0.001
n=9

Using GoldenRatio
golden_eps=0.01
eps=0.001
==== Results =====
Function results.
min: 5.37e-04, moda: 3.09e-03, avg: 3.09e-03, max: 1.09e-02
Function calculation. [2.76e+03 : 7.49e+06]

Using DSK-Powell
dsk_eps=0.0001
eps=1e-09
==== Results =====
Function results.
min: 7.80e-11, moda: 3.17e-10, avg: 3.17e-10, max: 2.21e-09
Function calculation. [4.70e+03 : 5.30e+04]
>
Using DSK-Powell method for optimal path allows us to get much more precise values
==== Testing how function calculation varies depending on starting point ====
d_swen=0.001
way_search=dsk
dsk_eps=0.0001
n=10
radius=0.001
eps=1e-09

Starting at (-100.00000, -100.00000)
min = 1.83e-11 with function calculated 4581 times
Starting at (-90.00000, -90.00000)
min = 8.94e-12 with function calculated 6749 times
Starting at (-80.00000, -80.00000)
min = 3.13e-11 with 

==== Results =====
Function results.
min: 1.14e-21, moda: 6.65e-17, avg: 6.65e-17, max: 6.34e-16
Function calculation. [8.41e+04 : 8.33e+05]
>
More tries produces slightly more precise values

== Testing how function calculation varies depending on epsilon in random search ==
d_swen=1e-05
start=(50.00000, 50.00000)
way_search=dsk
dsk_eps=0.0001
radius=0.001
n=10

Using eps=0.0001
==== Results =====
Function results.
min: 3.57e-03, moda: 3.32e+00, avg: 6.16e+01, max: 3.33e+02
Extremums.
min: (1.05975, 1.12301), max: (19.25954, 370.93548)
Function calculation. [4.66e+02 : 2.02e+04]
Using eps=1e-06
==== Results =====
Function results.
min: 2.98e-13, moda: 1.03e-08, avg: 3.94e+01, max: 3.94e+02
Extremums.
min: (0.99997, 0.99994), max: (20.84972, 434.71639)
Function calculation. [2.25e+03 : 2.42e+04]
Using eps=1e-09
==== Results =====
Function results.
min: 5.24e-20, moda: 1.23e-16, avg: 1.23e-16, max: 5.03e-16
Extremums.
min: (1.00000, 1.00000), max: (1.00000, 1.00000)
Function calculation

## Conclusions

Експереминатально були виявлені такі закономірності. \
Метод золотого перетину хоча і є простішим для імлементації та має деякі недоліки. По перше він викликає функцію доволі часто порівняно до методу ДСК-Пауела, і може призводити до шляхів нульової довжини що є умовою зупинки методу та призводить до false-positive помилок. Підвищуючи точність методу ми можемо досягти біль точнішу апроксимацію але це вимагає значно більших обчислень. Тому для всіх інших експериментів було використано метод ДСК-Пауела. \
Точність методу не залежить помітно від стартової точки або радіусу чи кількості випдакових точок. Якщо зменшити радіус більше ніж 0.01 метод видає приблизно однаково точні значення і так само з кількістю точок більше ніж 5. \
Метод збивається якщо стартує з (0, 0) але це можна обійти додавши невеликий шум до стартовоі точки. \
Параметр кроку у алгоритмі свена дає змогу підвищити точність методу і швидкість (у плані меншої кількості викликів функій) але при дуже малому кроці алгоритм призводить до генерації дуже малих шляхів і при шляху меншому ніж точність методу метод починає видавати false-positive. \
В цілому метод дозволяє отримувати доволі точні значення, наприклад для функції Розенброка це досягає 1e-19 з порівняно невеликою кількістью обчислень (менше 10 000).

Для умовної оптимізації метод все ще працює, звісно набагато швидше якщо локального мінімуму знаходиться всередині області, але і якщо за областю також, проте з набагато більшої кількістю обчислень.

## Authors

Денисенко Ілля <lightdarkdaughter@gmail.com> [Github](https://github.com/juliancoffee)

## References

"Методы одномерной оптимизации" http://www.e-biblio.ru/book/bib/02_estestv_nauki/Matem_Advance/Part9.html \
"Метод золотого сечения" https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B7%D0%BE%D0%BB%D0%BE%D1%82%D0%BE%D0%B3%D0%BE_%D1%81%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D1%8F